In [ ]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息準備
handler執行方法設計
啟動應用
'''

In [ ]:
'''
資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

colab 會把資料清空，在開發階段，為了保存資料，會希望掛載Google drive，把資料存在Google drive，只在開發階段這麼用，到時在GCP部署不會這樣用

'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''
流程解析：
引用套件
建置 app 主邏輯
生成準備傳給用戶的消息（以後可以把消息存在資料庫，提取出來）
設置以後收到消息要怎麼處理
運行app
'''


In [ ]:
# 安裝套件
!pip install line-bot-sdk flask flask-ngrok

In [ ]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [ ]:
'''
建置主程序app

建置handler與 line_bot_api
- handler：用來接收line 傳來的一切事務，順便做驗證，過濾信件
- line-bot-api：用來跟line溝通，回信、取個資
'''

# 設定Server啟用細節
app = Flask(__name__, static_url_path = "/material", static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("your channel access token")
handler = WebhookHandler("your channel secret")

In [ ]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發

當流量消息丟給ngrok.xxxx/，就會觸發line消息的處理流程
'''

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value，line的加密簽章，每一個line傳來的訊息都有一個，這是line設計的規則
    signature = request.headers['X-Line-Signature']

    # get request body as text，用戶傳來的內容
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log，存在Google drive上。在cloud shell裡面不會這樣用
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        # 把消息交給 handler 做驗證。會看是什麼樣的 Event，做不同的處理
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
'''
用戶關注就是一個 FollowEvent

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 回應用戶，歡迎用的文字消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

from linebot.models import(
    TextSendMessage, ImageSendMessage
)

# 告知handler，如果收到 FollowEvent，則做下面的方法處理。可以取個資、發消息、綁選單
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 用 line_bot_api 跟line溝通，取出消息內User的資料。event是每一次傳來的消息，source是來源，user_id是發消息的用戶id
    user_profile = line_bot_api.get_profile(event.source.user_id)   # 包含nickname，user_status, user_id, 大頭照，使用語系
        
     # 將用戶資訊存在檔案內
    with open("/content/drive/MyDrive/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')
    
    # 回覆文字消息與圖片消息。請line_bot_api回應消息給line，line轉發回去給用戶。
    # reply_token：每一次用戶穿消息給line，line轉傳過來時，都會附上這個臨時票券，我們必須拿這個票券跟line回應
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage('安安，你的個資已被我紀錄了')]
    )

In [ ]:
# 運行主程序
app.run()